In [ ]:
# --- BLOK 1: KURULUM VE ÖZELLİK ÇIKARIMI (Tek Hücrede) ---
print("Blok 1 Çalışıyor: Kurulum ve Özellik Çıkarımı Başladı (5-10 dk GPU)...")

# --- Kurulum ve Import ---
from google.colab import drive
drive.mount('/content/drive')
import zipfile, os, glob, time, json
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision import models
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA # PCA için eklendi

# --- Veri Hazırlığı (Blok 2'nin İçeriği) ---
zip_path = "/content/drive/MyDrive/archive (3).zip"
extract_path = "/content/cifake"
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(extract_path)
data_root = "/content/cifake"
train_dir = os.path.join(data_root, "train")
test_dir  = os.path.join(data_root, "test")

# DataFrame Oluşturma (kilitli_test_df ve egitim_alt_kume_df değişkenlerini oluşturur)
train_real = glob.glob(os.path.join(train_dir, "REAL", "*"))
train_fake = glob.glob(os.path.join(train_dir, "FAKE", "*"))
df_train = pd.DataFrame({ "path": train_real + train_fake, "label": [0]*len(train_real) + [1]*len(train_fake)})
df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)
egitim_alt_kume_df, _ = train_test_split(df_train, train_size=20000, random_state=42, stratify=df_train["label"])

test_real = glob.glob(os.path.join(test_dir, "REAL", "*"))
test_fake = glob.glob(os.path.join(test_dir, "FAKE", "*"))
df_test = pd.DataFrame({ "path": test_real + test_fake, "label": [0]*len(test_real) + [1]*len(test_fake)})
kilitli_test_df = df_test.copy()

# --- Özellik Çıkarımı (Blok 3'ün İçeriği) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform_resnet50 = T.Compose([ T.Resize((224, 224)), T.ToTensor(), T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
class ImageDataset(Dataset):
    def __init__(self, df, transform): self.df = df.reset_index(drop=True); self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        path = self.df.loc[idx, "path"]; label = int(self.df.loc[idx, "label"])
        try: img = Image.open(path).convert("RGB"); img = self.transform(img); return img, label
        except Exception as e: return torch.zeros((3, 224, 224)), -1
def collate_fn_safe(batch):
    batch = list(filter(lambda x: x[1] != -1, batch)); return torch.utils.data.dataloader.default_collate(batch)
resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2); resnet.fc = nn.Identity(); resnet = resnet.to(device); resnet.eval()
@torch.no_grad()
def extract_features(df, transform_to_use, batch_size=64):
    dataset = ImageDataset(df, transform_to_use); loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn_safe)
    feats, labels = [], []
    for imgs, labs in tqdm(loader, desc=f"Özellik Çıkarımı ({len(df)} görsel)"):
        if imgs.shape[0] == 0: continue
        imgs = imgs.to(device); f = resnet(imgs); feats.append(f.cpu().numpy()); labels.append(labs.numpy())
    return np.concatenate(feats, axis=0), np.concatenate(labels, axis=0)
X_train_all, y_train_all = extract_features(egitim_alt_kume_df, transform_resnet50)
X_test_locked, y_test_locked = extract_features(kilitli_test_df, transform_resnet50)
print(f"\n--- Blok 1 (Setup & Feature Extraction) Tamamlandı. X_train_all (2048 Özellik) hafızada. ---")

Blok 1 Çalışıyor: Kurulum ve Özellik Çıkarımı Başladı (5-10 dk GPU)...
Mounted at /content/drive
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 119MB/s]
Özellik Çıkarımı (20000 görsel): 100%|██████████| 313/313 [01:33<00:00,  3.34it/s]



--- Blok 1 (Setup & Feature Extraction) Tamamlandı. X_train_all (2048 Özellik) hafızada. ---


In [ ]:
# --- BLOK 2: PCA İLE BOYUT AZALTMA (FEATURE SELECTION) ---
print("Blok 2 Çalışıyor: PCA ile Boyut Azaltma (FS) Deneyi Başlıyor...")

# 1. PCA için StandardScaler kullanmak zorunludur
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_all)
X_test_locked_scaled = scaler.transform(X_test_locked)

# 2. Verideki Varyansın %95'ini koruyan bileşen sayısını bulma
print("Verideki varyansın %95'ini koruyacak temel bileşenler hesaplanıyor...")
pca = PCA(n_components=0.95)
pca.fit(X_train_scaled)

X_train_pca = pca.transform(X_train_scaled)
X_test_locked_pca = pca.transform(X_test_locked_scaled)

# 3. Sonuçları Raporlama
pca_bileşen_sayisi = pca.n_components_
print(f"Orijinal özellik boyutu (ResNet50): 2048")
print(f"PCA sonrası yeni özellik boyutu: {pca_bileşen_sayisi} (Varyansın %95'i)")
print("\n--- Blok 2 Tamamlandı. PCA değişkenleri hazır. ---")

Blok 2 Çalışıyor: PCA ile Boyut Azaltma (FS) Deneyi Başlıyor...
Verideki varyansın %95'ini koruyacak temel bileşenler hesaplanıyor...
Orijinal özellik boyutu (ResNet50): 2048
PCA sonrası yeni özellik boyutu: 1509 (Varyansın %95'i)

--- Blok 2 Tamamlandı. PCA değişkenleri hazır. ---


In [ ]:
# --- BLOK 3: FS KANITI İÇİN MODELLERİN EĞİTİMİ (ZAMAN/PERFORMANS) ---
# (Bu hücredeki SVM eğitimi yine 15-20 dk sürecektir)
print("Blok 3 Çalışıyor: PCA Uygulandı vs. Uygulanmadı Eğitimi (Zaman Kıyaslaması) Başlıyor...")

# 1. Kullanılacak Modeller (En yavaşlar ve baseline'lar)
models_fs = {
    "LogReg": LogisticRegression(max_iter=1000),
    "SVM_RBF": SVC(kernel="rbf", probability=True),
    "XGBoost": XGBClassifier(n_estimators=300, max_depth=6, random_state=42),
}

fs_results = []
X_train_original = X_train_all # Orijinal veriyi kullanalım

# 2. Deneyler
deneyler = [
    {"name": "Uygulanmadı (2048)", "train_data": X_train_scaled, "test_data": X_test_locked_scaled, "Bileşen Sayısı": 2048},
    {"name": f"PCA Uygulandı ({pca_bileşen_sayisi})", "train_data": X_train_pca, "test_data": X_test_locked_pca, "Bileşen Sayısı": pca_bileşen_sayisi}
]

for deney in deneyler:
    print(f"\n--- Deney Grubu: {deney['name']} ---")
    for name, model in models_fs.items():
        print(f"- {name} Eğitiliyor...")
        start_time = time.time()

        # SVM ve LogReg ölçeklenmiş veriye ihtiyaç duyar, XGBoost scale edilmiş veriyi kabul eder
        model.fit(deney['train_data'], y_train_all)
        end_time = time.time()

        # Tahmin yap
        y_pred = model.predict(deney['test_data'])
        f1 = f1_score(y_test_locked, y_pred, average="binary")

        fs_results.append({
            "Model": name,
            "FS Durumu": deney['name'],
            "Eğitim Süresi (s)": end_time - start_time,
            "F1-Skoru (%)": f1 * 100,
            "Bileşen Sayısı": deney['Bileşen Sayısı']
        })

# 3. Sonuçları DataFrame'e çevir
fs_df = pd.DataFrame(fs_results).round(2)

print("\n--- Blok 3 Tamamlandı. FS Analiz sonuçları hazır. ---")

Blok 3 Çalışıyor: PCA Uygulandı vs. Uygulanmadı Eğitimi (Zaman Kıyaslaması) Başlıyor...

--- Deney Grubu: Uygulanmadı (2048) ---
- LogReg Eğitiliyor...
- SVM_RBF Eğitiliyor...
- XGBoost Eğitiliyor...

--- Deney Grubu: PCA Uygulandı (1509) ---
- LogReg Eğitiliyor...
- SVM_RBF Eğitiliyor...
- XGBoost Eğitiliyor...

--- Blok 3 Tamamlandı. FS Analiz sonuçları hazır. ---


In [ ]:
# --- BLOK 4: FS KANITI İÇİN TABLO OLUŞTURMA VE KAPANIŞ ---
print("Blok 4 Çalışıyor: FS Kanıtı raporlanıyor...")

fs_rapor_dosyasi = "Hibrit_FS_Kaniti_Raporu.txt"

with open(fs_rapor_dosyasi, 'w') as f:
    f.write("===== ÖZNİTELİK SEÇİMİ (FS) KANITI (2048 vs. PCA) =====\n\n")
    f.write(f"PCA ile boyut, 2048'den {pca_bileşen_sayisi} bileşene düşürülmüştür (Varyansın %95'i korunmuştur).\n\n")

    # Tabloyu dosyaya yazdır
    f.write(fs_df.to_string(index=False))
    f.write("\n\n=======================================================\n")

    # Analiz ve Yorum (Hocanın istediği kritik kısım)
    svm_2048 = fs_df[(fs_df['Model'] == 'SVM_RBF') & (fs_df['Bileşen Sayısı'] == 2048)].iloc[0]
    svm_pca = fs_df[(fs_df['Model'] == 'SVM_RBF') & (fs_df['Bileşen Sayısı'] == pca_bileşen_sayisi)].iloc[0]

    time_reduction_percent = (1 - (svm_pca['Eğitim Süresi (s)'] / svm_2048['Eğitim Süresi (s)'])) * 100

    f.write(f"\nANALİZ VE YORUM (Yönerge Zorunluluğu - Bölüm 1.1):\n")
    f.write("-------------------------------------------------------\n")
    f.write(f"1. EĞİTİM SÜRESİ KAZANCI (SVM Modeli Üzerinden Kanıt):\n")
    f.write(f"   SVM Modeli, özellik boyutu %{100 - (pca_bileşen_sayisi/2048)*100:.2f} oranında ({2048} -> {pca_bileşen_sayisi}) azaltıldığında, eğitim süresi **%{time_reduction_percent:.2f}** oranında azalmıştır.\n")
    f.write(f"   (Örn: 2048 Özellik: {svm_2048['Eğitim Süresi (s)']:.2f} saniye vs. PCA Özellik: {svm_pca['Eğitim Süresi (s)']:.2f} saniye).\n")
    f.write(f"   Bu, Öznitelik Seçiminin (PCA), modelin karmaşıklığını ve dolayısıyla maliyetini somut olarak azalttığını kanıtlar.\n")
    f.write(f"2. PERFORMANS ETKİSİ:\n")
    f.write(f"   PCA uygulanan SVM modeli ({svm_pca['F1-Skoru (%)']:.2f}% F1) ile uygulanmayan model ({svm_2048['F1-Skoru (%)']:.2f}% F1) arasındaki F1-Skoru farkı minimum düzeydedir. Bu da, PCA'nın bilginin büyük kısmını başarıyla koruduğunu gösterir.\n")

print("\n--- Blok 4 Tamamlandı. FS Analizi raporu ve sonuçları hazırlandı. ---")

print("\n\n===== TÜM İŞLEMLER TAMAMLANDI =====")
print("Sol taraftaki 'Dosyalar' (Files) panelini yenileyerek 'Hibrit_FS_Kaniti_Raporu.txt' dosyanızı alabilirsiniz.")
!ls -l /content/Hibrit_FS_Kaniti_Raporu.txt

Blok 4 Çalışıyor: FS Kanıtı raporlanıyor...

--- Blok 4 Tamamlandı. FS Analizi raporu ve sonuçları hazırlandı. ---


===== TÜM İŞLEMLER TAMAMLANDI =====
Sol taraftaki 'Dosyalar' (Files) panelini yenileyerek 'Hibrit_FS_Kaniti_Raporu.txt' dosyanızı alabilirsiniz.
-rw-r--r-- 1 root root 1516 Nov 18 21:15 /content/Hibrit_FS_Kaniti_Raporu.txt
